In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re
import sqlite3

from nltk import word_tokenize 

In [2]:
pd.set_option('display.max_rows', 500, 'display.max_columns', 500)

In [3]:
# Import sales data.
sales = pd.read_csv('Data/Iowa_Liquor_Sales.csv')

C:\Users\JeongwooHan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Import products detail data.
product_df = pd.read_csv('Data/Iowa_Liquor_Products.csv')

In [5]:
# Making the sales data columns a bit more code friendly by removing spaces.
sales.columns = sales.columns.str.replace(' ','')
product_df.columns = product_df.columns.str.replace(' ','')

In [6]:
sales.columns

Index(['Invoice/ItemNumber', 'Date', 'StoreNumber', 'StoreName', 'Address',
       'City', 'ZipCode', 'StoreLocation', 'CountyNumber', 'County',
       'Category', 'CategoryName', 'VendorNumber', 'VendorName', 'ItemNumber',
       'ItemDescription', 'Pack', 'BottleVolume(ml)', 'StateBottleCost',
       'StateBottleRetail', 'BottlesSold', 'Sale(Dollars)',
       'VolumeSold(Liters)', 'VolumeSold(Gallons)'],
      dtype='object')

In [7]:
product_df.columns

Index(['ItemNumber', 'CategoryName', 'ItemDescription', 'Vendor', 'VendorName',
       'BottleVolume(ml)', 'Pack', 'InnerPack', 'Age', 'Proof', 'ListDate',
       'UPC', 'SCC', 'StateBottleCost', 'StateCaseCost', 'StateBottleRetail',
       'ReportDate'],
      dtype='object')

# Products Table

Clean up inconsistency by:
- Using category name that is most recent
- Using item description that is most recent

In [8]:
# Restrict to the fields we want for product table.
prod_col = ['Date',
           'Category',
           'CategoryName',
           'ItemNumber',
           'ItemDescription',
           'Pack',
           'BottleVolume(ml)']

product = sales[prod_col].copy()

product['CategoryName'] = product['CategoryName'].str.lower()
product['ItemDescription'] = product['ItemDescription'].str.lower()

In [9]:
product['ItemNumber'] = product['ItemNumber'].apply(str)

In [11]:
# Confirm that there are multiple category names associated with a single category ID. Will consolidate.
product.groupby(['Category', 'CategoryName']).agg({'Date':'count'})

Date
Category  CategoryName                                 
1011100.0 blended whiskies                       940814
1011200.0 straight bourbon whiskies             1303265
1011250.0 single barrel bourbon whiskies           9268
1011300.0 single barrel bourbon whiskies          16136
          tennessee whiskies                     245162
1011400.0 bottled in bond bourbon                  6417
          tennessee whiskies                     322660
1011500.0 bottled in bond bourbon                  9611
          straight rye whiskies                   59869
1011600.0 corn whiskies                             752
          straight rye whiskies                   82175
1011700.0 corn whiskies                            7975
1011800.0 iowa distillery whiskies                  124
1012000.0 imported whiskies                           7
1012100.0 canadian whiskies                     2090073
1012200.0 scotch whiskies                        390830
1012210.0 single malt scotch                      82675
1012300.0 irish whiskies                          88389
          single malt scotch                     101219
1012400.0 irish whiskies                         146777
          japanese whisky                           286
1022100.0 mixto tequila                          252961
          tequila                                435298
1022200.0 100% agave tequila                     329889
1022300.0 mezcal                                   6279
1031000.0 american vodka                           5223
1031080.0 vodka 80 proof                        1265974
1031090.0 other proof vodka                        3398
1031100.0 100 proof vodka                         99124
          american vodkas                       1747627
1031110.0 low proof vodka                          1565
1031200.0 american flavored vodka                577235
          vodka flavored                         502813
1032000.0 imported vodka                           1542
1032080.0 imported vodka                         404048
1032100.0 imported vodkas                        403706
1032200.0 imported flavored vodka                233745
          imported vodka - misc                  256241
1032230.0 imported vodka - cherry                    14
1041100.0 american dry gins                      492327
1041150.0 flavored gins                           12010
1041200.0 american sloe gins                       7002
          flavored gin                             8711
1041300.0 american sloe gins                       3819
1042000.0 imported gins                               8
1042100.0 imported dry gins                      239768
1051010.0 american grape brandies                243144
1051100.0 american brandies                      374101
          apricot brandies                        38509
1051110.0 blackberry brandies                     59714
1051120.0 cherry brandies                         13216
1051140.0 peach brandies                          15070
1051150.0 miscellaneous  brandies                  4214
1052010.0 imported grape brandies                155688
1052100.0 imported brandies                      244689
1062050.0 barbados rum                             9909
1062100.0 gold rum                                74753
          jamaica rum                             15288
1062200.0 puerto rico & virgin islands rum       395376
          white rum                              280408
1062300.0 aged dark rum                           33077
          flavored rum                           278367
1062310.0 spiced rum                             530323
1062400.0 spiced rum                             568193
1062500.0 flavored rum                           295568
1070000.0 cocktails / rtd                           948
1070100.0 temporary  & specialty packages            35
1071100.0 american cocktails                     284695
          cocktails /rtd                         334336
1081000.0 american cordials & liqueurs             2229
1081010.0 american 

In [12]:
# Create year field using the date field so we can use the most recent category name for each ID.
product['Year'] = product['Date'].str[-4:]
product['Year'].value_counts()

2020    2614365
2019    2380345
2018    2355558
2017    2291276
2016    2279893
2015    2184483
2014    2097796
2012    2082059
2013    2063763
2021    1291619
Name: Year, dtype: int64

In [13]:
# Use the most recent year's category name and consolidate it with category.
Category_List = product.sort_values(['Year']).drop_duplicates(subset = ['Category'], keep = 'last')
Merged_Product = product.merge(Category_List[['Category', 'CategoryName']], on = 'Category', how = 'left')
Merged_Product.drop(columns = 'CategoryName_x', inplace = True)
Merged_Product.rename(columns = {'CategoryName_y': 'CategoryName'}, inplace = True)
product = Merged_Product.copy()

In [14]:
# Check if category name cleanup worked as expected.
product.groupby(['Category', 'CategoryName']).agg({'Date':'count'})

,,Date
Category,CategoryName,
1011100.0,blended whiskies,940814
1011200.0,straight bourbon whiskies,1303265
1011250.0,single barrel bourbon whiskies,9268
1011300.0,single barrel bourbon whiskies,261298
1011400.0,tennessee whiskies,329077
1011500.0,bottled in bond bourbon,69480
1011600.0,straight rye whiskies,82927
1011700.0,corn whiskies,8014
1011800.0,iowa distillery whiskies,124


In [15]:
# Use the most recent year's item description and consolidate it with item number.
Item_List = product.sort_values(['Year']).drop_duplicates(subset = ['ItemNumber'], keep = 'last')
Merged_Item = product.merge(Item_List[['ItemNumber', 'ItemDescription']], on = 'ItemNumber', how = 'left')
Merged_Item.drop(columns = 'ItemDescription_x', inplace = True)
Merged_Item.rename(columns = {'ItemDescription_y': 'ItemDescription'}, inplace = True)
product = Merged_Item.copy()

In [16]:
product.head()

,Date,Category,ItemNumber,Pack,BottleVolume(ml),Year,CategoryName,ItemDescription
0,03/16/2016,1081200.0,80457,12,1000,2016,cream liqueurs,ryans cream liqueur
1,07/15/2019,1012200.0,10008,6,1750,2019,scotch whiskies,scoresby rare scotch
2,07/01/2020,1012400.0,15628,6,1750,2020,irish whiskies,jameson
3,07/11/2019,1012200.0,10009,12,1000,2019,scotch whiskies,scoresby rare scotch
4,07/23/2019,1701100.0,100107,6,1850,2019,temporary & specialty packages,captain morgan osr glass w/50ml apple smash & ...


In [17]:
# Keep the bottle volume that is the most common.
bottle_volume = product.groupby(['ItemNumber']).agg({'BottleVolume(ml)':'value_counts'})\
                .rename(columns = {'BottleVolume(ml)':'count'}).reset_index()
bottle_volume['rank'] = bottle_volume.groupby(['ItemNumber'])['count'].rank(ascending = False)
bottle_volume_unique = bottle_volume[bottle_volume['rank'] == 1]

In [18]:
# Merge the unique volume list onto product table.
Merged_Volume = product.merge(bottle_volume_unique[['ItemNumber', 'BottleVolume(ml)']], on = 'ItemNumber', how = 'left')
Merged_Volume.drop(columns = 'BottleVolume(ml)_x', inplace = True)
Merged_Volume.rename(columns = {'BottleVolume(ml)_y': 'BottleVolume'}, inplace = True)
product = Merged_Volume.copy()

In [19]:
product = product[['ItemNumber', 'ItemDescription', 'Category', 'CategoryName', 'BottleVolume']]
product.head()

,ItemNumber,ItemDescription,Category,CategoryName,BottleVolume
0,80457,ryans cream liqueur,1081200.0,cream liqueurs,1000.0
1,10008,scoresby rare scotch,1012200.0,scotch whiskies,1750.0
2,15628,jameson,1012400.0,irish whiskies,1750.0
3,10009,scoresby rare scotch,1012200.0,scotch whiskies,1000.0
4,100107,captain morgan osr glass w/50ml apple smash & ...,1701100.0,temporary & specialty packages,1850.0


In [20]:
# Make the product table a unique list.
product.drop_duplicates(keep = 'first', inplace = True)
product.shape

(13338, 5)

In [21]:
product_copy = product.copy()

In [22]:
proof = product_df[['ItemNumber', 'Proof']]

In [23]:
# Making ItemNumber in proof database to be str to match onto product database.
proof['ItemNumber'] = proof['ItemNumber'].apply(str)

<ipython-input-23-3943bf8c521d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proof['ItemNumber'] = proof['ItemNumber'].apply(str)


In [27]:
# Attaching proof to the product database.
Proof_Merge = product_copy.merge(proof[['ItemNumber', 'Proof']], on = 'ItemNumber', how = 'left')
Proof_Merge.head()

,ItemNumber,ItemDescription,Category,CategoryName,BottleVolume,Proof
0,80457,ryans cream liqueur,1081200.0,cream liqueurs,1000.0,34.0
1,10008,scoresby rare scotch,1012200.0,scotch whiskies,1750.0,80.0
2,15628,jameson,1012400.0,irish whiskies,1750.0,80.0
3,10009,scoresby rare scotch,1012200.0,scotch whiskies,1000.0,80.0
4,100107,captain morgan osr glass w/50ml apple smash & ...,1701100.0,temporary & specialty packages,1850.0,70.0


In [28]:
product = Proof_Merge

In [29]:
product_groupcat = product.copy()

In [30]:
product_groupcat['ItemDescription'] = product_groupcat['ItemDescription'].apply(str)
product_groupcat['CategoryName'] = product_groupcat['CategoryName'].apply(str)

In [31]:
# Create category grouping.
product_groupcat['CategoryGroup'] = np.where(product_groupcat['CategoryName'].str.contains('rum'),'rum',
                                    np.where(product_groupcat['CategoryName'].str.contains('gin'),'gin', 
                                    np.where(product_groupcat['CategoryName'].str.contains('cocktail'), 'cocktail',
                                    np.where(product_groupcat['CategoryName'].str.contains('brandy|brandies'), 'brandy',
                                    np.where(product_groupcat['CategoryName'].str.contains('whisk|bourbon|scotch'), 'whisky',
                                    np.where(product_groupcat['CategoryName'].str.contains('tequila|mezcal'), 'tequila',
                                    np.where(product_groupcat['CategoryName'].str.contains('schnapps'), 'schnapps',
                                    np.where(product_groupcat['CategoryName'].str.contains('vodka'), 'vodka',
                                    np.where(product_groupcat['CategoryName'].str.contains('spirit'), 'spirit',
                                    np.where(product_groupcat['CategoryName'].str.contains('liqueur'), 'liqueur',
                                    np.where(product_groupcat['CategoryName'].str.contains('brandies'), 'brandy', 'other')))))))))))

In [32]:
product_groupcat['CategoryGroup'].value_counts()

whisky      3243
other       2312
vodka       2288
liqueur     1451
rum         1000
tequila      825
brandy       644
spirit       437
cocktail     412
gin          369
schnapps     357
Name: CategoryGroup, dtype: int64

In [33]:
# Confirm the categories left over can be categorized to "other"
product_groupcat[product_groupcat['CategoryGroup'] == 'other']['CategoryName'].value_counts()

special order items                892
temporary & specialty packages     837
nan                                355
temporary &  specialty packages     90
american amaretto                   23
holiday vap                         19
triple sec                          16
delisted items                      16
american alcohol                    13
white creme de cacao                10
creme de almond                      7
dark creme de cacao                  7
green creme de menthe                6
imported amaretto                    6
anisette                             3
iowa distilleries                    3
white creme de menthe                3
high proof beer - american           2
amaretto - imported                  2
temporary  & specialty packages      1
delisted / special order items       1
Name: CategoryName, dtype: int64

In [34]:
product_groupcat.head()

,ItemNumber,ItemDescription,Category,CategoryName,BottleVolume,Proof,CategoryGroup
0,80457,ryans cream liqueur,1081200.0,cream liqueurs,1000.0,34.0,liqueur
1,10008,scoresby rare scotch,1012200.0,scotch whiskies,1750.0,80.0,whisky
2,15628,jameson,1012400.0,irish whiskies,1750.0,80.0,whisky
3,10009,scoresby rare scotch,1012200.0,scotch whiskies,1000.0,80.0,whisky
4,100107,captain morgan osr glass w/50ml apple smash & ...,1701100.0,temporary & specialty packages,1850.0,70.0,other


In [35]:
product = product_groupcat[['ItemNumber', 'ItemDescription', 'Category', 'CategoryName', 'CategoryGroup', 'BottleVolume', 'Proof']]

In [36]:
# Clean up names so that it's consistent between tables.
product.rename(columns = {'ItemNumber':'ProductID', 'ItemDescription':'ProductName', 'Category':'CategoryID'}, inplace = True)
product.head()

,ProductID,ProductName,CategoryID,CategoryName,CategoryGroup,BottleVolume,Proof
0,80457,ryans cream liqueur,1081200.0,cream liqueurs,liqueur,1000.0,34.0
1,10008,scoresby rare scotch,1012200.0,scotch whiskies,whisky,1750.0,80.0
2,15628,jameson,1012400.0,irish whiskies,whisky,1750.0,80.0
3,10009,scoresby rare scotch,1012200.0,scotch whiskies,whisky,1000.0,80.0
4,100107,captain morgan osr glass w/50ml apple smash & ...,1701100.0,temporary & specialty packages,other,1850.0,70.0


In [37]:
product.shape

(13338, 7)

In [40]:
# Going to leave nulls here since we'll most likely be focusing on category name anyways.
product.isnull().sum(axis = 0)

ProductID           0
ProductName         0
CategoryID        351
CategoryName        0
CategoryGroup       0
BottleVolume       13
Proof            1567
dtype: int64

In [41]:
product.loc[product['CategoryName'] == 'nan']

,ProductID,ProductName,CategoryID,CategoryName,CategoryGroup,BottleVolume,Proof
705,984393,sooh 99 watermelon mini,NaN,nan,other,600.0,99.0
1478,80024,blue chair bay banana rum cream,NaN,nan,other,750.0,30.0
1537,920376,sooh old forester 1920,NaN,nan,other,750.0,115.0
1756,916839,black saddle bourbon,NaN,nan,other,750.0,90.0
1829,985556,koval rye,NaN,nan,other,750.0,80.0
2039,995509,bains cape mountain whiskey,NaN,nan,other,750.0,86.0
2107,994046,sooh aberlour a'bunadh,NaN,nan,other,750.0,122.0
2197,36122,deep eddy 80prf,NaN,nan,other,750.0,80.0
2250,40569,river pilot strawberry vodka,NaN,nan,other,375.0,80.0
2304,457,makers mark co-pack,NaN,nan,other,750.0,90.0


In [135]:
product.to_csv('Data/product_final.csv', index_label = False)

# Product Price Table

In [43]:
# Restrict to the fields we want for product price table.
prod_price_col = ['ItemNumber',
                  'Date',
                  'StateBottleRetail',
                  'StateBottleCost']

product_price = sales[prod_price_col].copy()

In [44]:
product_price['Date'] = pd.to_datetime(product_price['Date'])

In [45]:
# Use the mean of StateBottleRetail and StateBottleCost assuming that it is consistent for that specific day if there were many.
product_price = product_price.groupby(['ItemNumber', 'Date']).agg({'StateBottleRetail':'mean', 'StateBottleCost':'mean'})\
                .reset_index()

In [46]:
# Clean up names so that it's consistent between tables.
product_price.rename(columns = {'ItemNumber': 'ProductID'}, inplace = True)

In [47]:
product_price.head()

,ProductID,Date,StateBottleRetail,StateBottleCost
0,101,2016-10-13,29.97,19.98
1,101,2016-10-17,29.97,19.98
2,101,2016-10-18,29.97,19.98
3,101,2016-10-19,29.97,19.98
4,101,2016-10-26,29.97,19.98


In [48]:
product_price.isnull().sum(axis = 0)

ProductID            0
Date                 0
StateBottleRetail    3
StateBottleCost      3
dtype: int64

In [49]:
product_price.loc[product_price['StateBottleRetail'].isnull()]

,ProductID,Date,StateBottleRetail,StateBottleCost
1160241,35920,2012-01-31,NaN,NaN
1375942,38180,2012-01-30,NaN,NaN
1375943,38180,2012-01-31,NaN,NaN


In [50]:
# Removing nulls
product_price.dropna(inplace = True)
product_price.isnull().sum(axis = 0)

ProductID            0
Date                 0
StateBottleRetail    0
StateBottleCost      0
dtype: int64

In [51]:
product_price.to_csv('Data/product_price_final.csv', index_label = False)

# Vendors Table

Clean up inconsistency by:
- Using category name that is most recent
- Using item description that is most recent

In [52]:
# Restrict to the fields we want for vendor table.
vendor_col = ['Date',
              'VendorNumber',
              'VendorName']

vendor = sales[vendor_col].copy()

In [53]:
vendor['Year'] = vendor['Date'].str[-4:]

In [54]:
# Use the most recent year's vendor name and consolidate it with vendor number.
Vendor_List = vendor.sort_values(['Year']).drop_duplicates(subset = ['VendorNumber'], keep = 'last')
Merged_Vendor = vendor.merge(Vendor_List[['VendorNumber', 'VendorName']], on = 'VendorNumber', how = 'left')
Merged_Vendor.drop(columns = 'VendorName_x', inplace = True)
Merged_Vendor.rename(columns = {'VendorName_y': 'VendorName'}, inplace = True)

In [55]:
vendor = Merged_Vendor[['VendorNumber', 'VendorName']]
vendor.drop_duplicates(inplace = True)

<ipython-input-55-446e20498080>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vendor.drop_duplicates(inplace = True)


In [56]:
vendor.shape

(378, 2)

In [57]:
# Clean up names so that it's consistent between tables.
vendor.rename(columns = {'VendorNumber': 'VendorID'}, inplace = True)

C:\Users\JeongwooHan\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [58]:
vendor.head()

,VendorID,VendorName
0,330.0,Gemini Spirits
1,421.0,SAZERAC COMPANY INC
2,370.0,PERNOD RICARD USA
4,260.0,DIAGEO AMERICAS
7,410.0,Patron Spirits Company


In [59]:
vendor.isnull().sum(axis = 0)

VendorID      1
VendorName    1
dtype: int64

In [60]:
vendor.loc[vendor['VendorID'].isnull()]

,VendorID,VendorName
4671186,NaN,NaN


In [61]:
# Removing nulls
vendor.dropna(inplace = True)
vendor.isnull().sum(axis = 0)

<ipython-input-61-a1a30061e738>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vendor.dropna(inplace = True)


VendorID      0
VendorName    0
dtype: int64

In [62]:
vendor.to_csv('Data/vendor_final.csv', index_label = False)

# Stores Table

Clean up inconsistency by:
- Standardizing the store names
- Split store location into 'long' and 'lat'

In [104]:
# Restrict to the fields we want for store table.
store_col = ['StoreNumber',
             'StoreName',
             'City',
             'ZipCode',
             'StoreLocation',
             'CountyNumber',
             'County']

store = sales[store_col].drop_duplicates(store_col, keep = 'first').copy()

store.dropna(inplace = True)

In [105]:
# When I tried to convert zip code to integer, it brought up the issue that there is '712-2'.
# I looked into it, and it seems to be for dunlap, which other stores have zip codes '51529'. So will be replacing that.
store.loc[store['ZipCode'] == '712-2','ZipCode'] = '51529'
store.loc[store['ZipCode'] == '51529']

,StoreNumber,StoreName,City,ZipCode,StoreLocation,CountyNumber,County
2393660,4307,Crossroads Wine and Liquor,DUNLAP,51529,POINT (-95.602969 41.854728),43.0,Harrison
4785147,5362,Crossroads Wine & Spirits LLC,Dunlap,51529,POINT (-95.602969 41.854728),43.0,HARRISON
4813654,4307,Crossroads Wine and Liquor,Dunlap,51529,POINT (-95.602969 41.854728),43.0,Harrison


In [106]:
store['ZipCode'] = store['ZipCode'].apply(int).round().apply(str)

In [108]:
store['StoreName'] = store['StoreName'].apply(lambda x: x.split('/')[0].strip())

In [109]:
# Formula to standardize names.
def standardize_name(name):
    name = str(name)
    name = name.replace("``",'')
    name = name.replace('"','')
    name = re.sub("[,`'#.]",'',name)
    name = str(name).lower().replace(' and ',' & ')
    store_name = word_tokenize(name)
    store_name = [str(i).capitalize().strip() for i in store_name]
    store_name = ' '.join(store_name)
    store_name = store_name.replace('  ',' ')
    return store_name

In [110]:
# Applying standarization to StoreName, City, and County.
for col in ['StoreName', 'City', 'County']:
    store[col] = store[col].apply(standardize_name)

In [111]:
# Creating a list for StoreLocation
def get_location(location):
    if str(location) != 'nan':
        location = str(location)
        location = re.findall(r'[-]\d.*\d', location)[0]
        lon, lat = location.split(' ')
        lon = round(float(lon), 2)
        lat = round(float(lat), 2)
        location = [lon, lat]
    else:
        location = np.nan
    return location

In [112]:
store['StoreLocation'] = store['StoreLocation'].map(get_location)
store

,StoreNumber,StoreName,City,ZipCode,StoreLocation,CountyNumber,County
9,2663,Hy-vee Food Store,Urbandale,50322,"[-93.74, 41.63]",77.0,Polk
18,2647,Hy-vee 7,Cedar Rapids,52411,"[-91.7, 42.03]",57.0,Linn
24,5259,Gameday Liquor,Glenwood,51534,"[-95.92, 41.28]",65.0,Mills
27,5597,Liquor Tobacco & Gas,Dubuque,52001,"[-90.67, 42.51]",31.0,Dubuque
31,3749,Wal-mart 1526,Storm Lake,50588,"[-95.2, 42.66]",11.0,Buena Vist
...,...,...,...,...,...,...,...
21550279,6055,Caseys General Store 1428,Milo,50166,"[-93.44, 41.29]",91.0,Warren
21570146,6172,Bp To Go,Hiawatha,52233,"[-91.67, 42.05]",57.0,Linn
21587940,6179,Bp To Go - 1010,Marion,52302,"[-91.6, 42.02]",57.0,Linn
21625528,6176,Mini-mart,Cedar Falls,50613,"[-92.46, 42.54]",7.0,Black Hawk


In [113]:
# Dividing the StoreLocation list to Long and Lat.
long_lst = []
lat_lst = []
for i in store['StoreLocation']:
    long, lat = str(i).split(', ')
    long_lst.append(float(long.replace('[','')))
    lat_lst.append(float(lat.replace(']','')))
    
store['long'] = long_lst
store['lat'] = lat_lst

In [114]:
store.drop('StoreLocation', axis = 1, inplace = True)

In [115]:
store.isnull().sum(axis = 0)

StoreNumber     0
StoreName       0
City            0
ZipCode         0
CountyNumber    0
County          0
long            0
lat             0
dtype: int64

In [116]:
# Clean up names so that it's consistent between tables.
store.rename(columns = {'StoreNumber':'StoreID', 'long':'Long', 'lat':'Lat'}, inplace = True)

In [117]:
store.head()

,StoreID,StoreName,City,ZipCode,CountyNumber,County,Long,Lat
9,2663,Hy-vee Food Store,Urbandale,50322,77.0,Polk,-93.74,41.63
18,2647,Hy-vee 7,Cedar Rapids,52411,57.0,Linn,-91.70,42.03
24,5259,Gameday Liquor,Glenwood,51534,65.0,Mills,-95.92,41.28
27,5597,Liquor Tobacco & Gas,Dubuque,52001,31.0,Dubuque,-90.67,42.51
31,3749,Wal-mart 1526,Storm Lake,50588,11.0,Buena Vist,-95.20,42.66


In [118]:
store.drop_duplicates(keep = 'first', inplace = True)

In [119]:
store

,StoreID,StoreName,City,ZipCode,CountyNumber,County,Long,Lat
9,2663,Hy-vee Food Store,Urbandale,50322,77.0,Polk,-93.74,41.63
18,2647,Hy-vee 7,Cedar Rapids,52411,57.0,Linn,-91.70,42.03
24,5259,Gameday Liquor,Glenwood,51534,65.0,Mills,-95.92,41.28
27,5597,Liquor Tobacco & Gas,Dubuque,52001,31.0,Dubuque,-90.67,42.51
31,3749,Wal-mart 1526,Storm Lake,50588,11.0,Buena Vist,-95.20,42.66
...,...,...,...,...,...,...,...,...
21550279,6055,Caseys General Store 1428,Milo,50166,91.0,Warren,-93.44,41.29
21570146,6172,Bp To Go,Hiawatha,52233,57.0,Linn,-91.67,42.05
21587940,6179,Bp To Go - 1010,Marion,52302,57.0,Linn,-91.60,42.02
21625528,6176,Mini-mart,Cedar Falls,50613,7.0,Black Hawk,-92.46,42.54


In [121]:
store.to_csv('Data/store_final.csv', index_label = False)

# Transactions Table

In [122]:
# Restrict to the fields we want for store table.
transaction_col = ['Invoice/ItemNumber',
                   'StoreNumber',
                   'VendorNumber',
                   'ItemNumber',
                   'Date',
                   'BottlesSold',
                   'VolumeSold(Gallons)',
                   'Sale(Dollars)']

transaction = sales[transaction_col].copy()

In [123]:
transaction['Date'] = pd.to_datetime(transaction['Date'])

In [124]:
# Clean up names so that it's consistent between tables.
transaction.rename(columns = {'Invoice/ItemNumber':'TransactionID',
                              'StoreNumber':'StoreID',
                              'VendorNumber':'VendorID',
                              'ItemNumber':'ProductID',
                              'VolumeSold(Gallons)':'Volume',
                              'Sale(Dollars)':'Sale'}, inplace = True)

In [125]:
transaction.isnull().sum(axis = 0)

TransactionID     0
StoreID           0
VendorID          9
ProductID         0
Date              0
BottlesSold       0
Volume            0
Sale             10
dtype: int64

In [132]:
# Removing nulls since there are only 10 rows out of 2M+ rows.
transaction.dropna(inplace = True)
transaction.isnull().sum(axis = 0)

TransactionID    0
StoreID          0
VendorID         0
ProductID        0
Date             0
BottlesSold      0
Volume           0
Sale             0
dtype: int64

In [133]:
transaction.shape

(21641138, 8)

In [134]:
transaction.to_csv('Data/transaction_final.csv', index_label = False)

# Set Up SQLite Database

In [137]:
con = sqlite3.connect('IowaLiquor.db')
cur = con.cursor()

In [138]:
cur.execute('''CREATE TABLE Product
                (ProductID, ProductName, CategoryID, CategoryName, CategoryGroup, BottleVolume, Proof)''')

cur.execute('''CREATE TABLE Price
                (ProductID, Date, StateBottleRetail, StateBottleCost)''')

cur.execute('''CREATE TABLE Venor
                (VendorID, VendorName)''')

cur.execute('''CREATE TABLE Store
                (StoreID, StoreName, City, ZipCode, CountyNumber, County, Long, Lat)''')

cur.execute('''CREATE TABLE Transactions
                (TransactionID, StoreID, VendorID, ProductID, Date, BottlesSold, Volume, Sale)''')

In [139]:
product.to_sql('Product', con, if_exists = 'append', index = False)
product_price.to_sql('Price', con, if_exists = 'append', index = False)
vendor.to_sql('Vendor', con, if_exists = 'append', index = False)
store.to_sql('Store', con, if_exists = 'append', index = False)
transaction.to_sql('Transactions', con, if_exists = 'append', index = False)

In [140]:
con.commit()

In [141]:
con.close()